In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [2]:
with open("data/training_data.jsonl", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded file ID:", file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded file ID:", file_obj.id)

Uploaded file ID: file-9w8udqP6kf1YHS7tp3gY2e
Uploaded file ID: file-P9u7rdEMsdS9g6eEyhBvwQ


In [4]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file="file-9w8udqP6kf1YHS7tp3gY2e",       # The file ID you uploaded
    validation_file="file-P9u7rdEMsdS9g6eEyhBvwQ",     # Optional
    model="gpt-4.1-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-voice-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

Fine-tune job created: ftjob-wQHQ0kPVGlG1gtiL9Atuqpw5


In [5]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-wQHQ0kPVGlG1gtiL9Atuqpw5 validating_files None


In [7]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-2025-04-14:personal:brand-voice-support:CPjcskrj",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  